## Compile Projects
To-do
* Figure out how to version things b/c projects will get updated and we want to track any changes.
* This only needs to be done with data from lp2000 and ctips.
* Need to track changes across all the different dataframes
* Use merges to figure it out?

In [1]:
import _harmonization_utils as har_utils
import _lrtp_utils as lrtp_utils
import _sb1_utils as sb1_utils
import _specific_list_utils
import _state_rail_plan_utils as srp_utils
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
import hashlib
from datetime import datetime

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_list/"

### LRTP/LOST

In [4]:
lrtp_lost_df, lrtp_lost_gdf = lrtp_utils.all_mpo(True)

/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

96 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:720: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sandag.cost2020m.str.replace("$", "")
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


65 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


360 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


68 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

In [5]:
def database_unique_project_number(df:pd.DataFrame, project_id_col:str, database:str) -> pd.DataFrame:
   
    # df['combo'] = df.apply(lambda row: f"{row.name}{row['timestamp']}{''.join(map(str, row))}", axis=1)
    # df['project_number'] = df['combo'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:12])
    # df = df.drop(columns = ['combo', 'timestamp'])
    
    df['project_number'] = df[project_id_col].astype(str) + df[database].str.replace(' ', '_')
    # Pad the number
    df['project_number'] = df['project_number'].str.pad(width=14, fillchar='0')
    
    # df = df.drop(columns = [project_id_col])
    return df

In [6]:
def non_database_unique_project_number(df:pd.DataFrame, data_source_col:str) -> pd.DataFrame:
   
    df['project_count'] = df.groupby([data_source_col]).cumcount().astype(int)+1
    # Pad the number
    df['project_number'] = df[data_source_col].str.replace(' ', '_') + df.project_count.astype(str)
    
    df.project_number = df.project_number.str.pad(width=14, fillchar='0')
    
    df = df.drop(columns = ['project_count'])
    return df

In [7]:
def separate_out_df(df:pd.DataFrame, columns_to_keep: list)-> pd.DataFrame:
    """
    Subset the column, drop any rows 
    in which the values are Nan or "None."
    """
    # Subset
    df2 = df[columns_to_keep]
    
    # Fill in missing values
    try:
        df2 = df2.fillna('none')
    except:
        df2
    
    # Remove project_number and keep only cols of interest
    columns_to_keep.remove('project_number')
    
    # Drop rows that are nan or "None" based on how many columns are listed
    if len(columns_to_keep) == 1:
        df2 = df2.dropna(how = "any")
        df2 = df2[df2.applymap(lambda x: x.lower() if isinstance(x, str) else x) != 'none'].dropna()
        
    # If there are more than one column to separate out,
    # keep any row that has a non-null value 
    else:
        df2 = df2.dropna(how = "all", subset = columns_to_keep)
    return df2

In [8]:
lrtp_lost_df = non_database_unique_project_number(lrtp_lost_df, 'data_source')

In [9]:
lrtp_lost_df.sample()

,project_title,lead_agency,project_year,project_description,total_project_cost,geometry,city,county,data_source,notes,project_number
805,Kern County ‐ Class Iii Bike Blvd,Kern Cog,None,Habecker Rd (Panama Rd To Di Giorgio Rd) 1. Mi. $ 50,50000.00,None,None,None,Kern Cog Lrtp,"Yoe W New Revenue: 50000, Yoe W O New Reven: 50000",Kern_Cog_Lrtp806


In [10]:
lrtp_lost_county = separate_out_df(lrtp_lost_df, ['project_number', 'county'])

In [11]:
lrtp_lost_county.sample()

,project_number,county
146,000Mtc_Lrtp147,Various


In [12]:
lrtp_lost_city = separate_out_df(lrtp_lost_df, ['project_number', 'city'])

In [13]:
lrtp_lost_city.shape

(745, 2)

In [14]:
lrtp_lost_agency = separate_out_df(lrtp_lost_df, ['project_number', 'lead_agency'])

In [15]:
lrtp_lost_agency.sample()

,project_number,lead_agency
19,00000000Lost20,Lost


In [16]:
lrtp_lost_geo = separate_out_df(lrtp_lost_df, ['project_number', 'geometry'])

In [17]:
lrtp_lost_geo.shape, lrtp_lost_gdf.shape

((1357, 2), (1355, 10))

In [18]:
lrtp_to_drop = ['county', 'city', 'lead_agency', 'geometry']

In [19]:
lrtp_lost_df = lrtp_lost_df.drop(columns = lrtp_to_drop)

In [20]:
lrtp_lost_df.sample()

,project_title,project_year,project_description,total_project_cost,data_source,notes,project_number
47,Placerville Dr Bridge Widening,2020-2025,"Hangtown Creek Bridge At Placerville Drive, 0.3 Mi West Of Cold Springs Rd: Replace Existing Functionally Obsolete 2-Lane Bridge With A New 4-Lane Bridge.",4935550.00,Sacog Lrtp,"Budget Category: B- Road & Highway Capacity, Year Of Expenditure Cost For Planned Projects: Nan, Status Planned, Programmed Or Project Development Only : Programmed",00Sacog_Lrtp48


### SB1

In [21]:
sb1_df = sb1_utils.load_sb1()

/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gdf[i]
/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  gdf[i]


0 rows are mising geometry
7917 rows contain invalid geography


In [22]:
#  sb1_df['data_source'] = "SB1 Server"

In [23]:
sb1_df = database_unique_project_number(sb1_df, 'projectid', 'data_source')

In [24]:
sb1_df.sample()

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,description,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,cityname,countycode,citycode,appagencyname,impagencyname,geometry,totalcosts,routes,constyear,costfull,projagency,data_source,project_number
2286,LsrFy17185085Pp001,Roadway Scrub Seal 1,Local,Local And Regional,Lsr1718,201718 Local Streets And Roads,NaN,PreConstruction,Scrub Seal Roadway Rehabilitation Project,NaN,21,12,21,12,Merced,Merced Submitted By City,Mer,Mer,NaN,NaN,None,NaN,NaN,2018,0.00,Merced City,SB1 Server,LsrFy17185085Pp001SB1_Server


In [25]:
len(sb1_df)

9186

In [26]:
sb1_df.project_number.nunique()

7603

#### Check that assemblydistrict and assemblycode are the same values

In [27]:
sb1_df['assembly_same'] = sb1_df.assemblycode == sb1_df.assemblydistrict

In [28]:
sb1_df.assembly_same.value_counts()

True     7217
False    1969
Name: assembly_same, dtype: int64

In [29]:
sb1_df['senate_same'] = sb1_df.senatedistrict == sb1_df.senatecode

In [30]:
sb1_df.senate_same.value_counts()

True     6950
False    2236
Name: senate_same, dtype: int64

In [31]:
sb1_county = separate_out_df(sb1_df, ['project_number', 'countyname'])

In [32]:
def explode_dataframe(df:pd.DataFrame, column_to_explode:str)-> pd.DataFrame:
    df['Column2'] = df[column_to_explode].apply(lambda x: [int(i) if i.isdigit() else i for i in x.replace(',', '').split()])
    df = df.drop(columns = [column_to_explode])
    df2 = df.explode('Column2')
    df2 = df2.rename(columns = {'Column2': column_to_explode})
    return df2

In [33]:
sb1_assembly = separate_out_df(sb1_df, ['project_number', 'assemblydistrict'])

In [34]:
sb1_assembly = explode_dataframe(sb1_assembly, 'assemblydistrict')

In [35]:
sb1_senate = separate_out_df(sb1_df, ['project_number', 'senatedistrict'])

In [36]:
sb1_senate = explode_dataframe(sb1_senate, 'senatedistrict')

In [37]:
sb1_city = separate_out_df(sb1_df, ['project_number', 'cityname'])

In [38]:
sb1_geo = separate_out_df(sb1_df, ['project_number', 'geometry'])

In [39]:
sb1_awards = separate_out_df(sb1_df, ['project_number', 'projprogram'])

In [40]:
sb1_awards.shape, sb1_geo.shape, sb1_city.shape

((9186, 2), (1585, 2), (6696, 2))

In [41]:
sb1_agencies = separate_out_df(sb1_df, ['project_number', 'projagency', 'appagencyname', 'impagencyname'])

In [42]:
sb1_agencies.shape

(9186, 4)

### LP2000

In [43]:
def load_lp2000(file: str):
    
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    ).drop(columns=["project_label_name"])

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    ).drop(columns=["project_label_name"])

    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    df_project['data_source'] = "lp2000"
    # Add timestamp column 
    return df_project, df_county, df_district, df_award, df_phase

In [44]:
lp2000_project, lp2000_county, lp2000_district, lp2000_award, lp2000_phase = load_lp2000("LP2000_projects.xlsx")

In [45]:
lp2000_project = database_unique_project_number(lp2000_project, 'project_id', 'data_source')

In [46]:
len(lp2000_project), lp2000_project.project_number.nunique()

(11272, 11272)

In [47]:
def add_project_number(df_with_project_num:pd.DataFrame, target_df:pd.DataFrame, merge_col:str)-> pd.DataFrame:
    df_with_project_num_subset = df_with_project_num[['project_number'] + [merge_col]]
    m1 = pd.merge(df_with_project_num_subset, target_df, on = merge_col, how = 'inner')
    m1 = m1.drop(columns = merge_col)
    return m1

In [48]:
lp2000_county = add_project_number(lp2000_project, lp2000_county, 'project_id')

In [49]:
lp2000_county.sample()

,project_number,county_name
7872,5079(032)lp2000,San Diego County


In [51]:
lp2000_district = add_project_number(lp2000_project, lp2000_district, 'project_id')

In [52]:
lp2000_district.sample()

,project_number,district_code
8080,5137(055)lp2000,4.00


In [53]:
lp2000_award = add_project_number(lp2000_project, lp2000_award, 'project_id')

In [54]:
lp2000_award.sample()

,project_number,grant_program,state_fiscal_awarded_year
900,5132(047)lp2000,Active Transportation Program (ATP),1819


In [55]:
lp2000_phase = add_project_number(lp2000_project, lp2000_phase, 'project_id')

In [56]:
lp2000_phase.sample()

,project_number,single_phase_cost,single_phase_expenditure_amt,total_state_funds,total_federal_funds,is_state,is_federal
1172,5940(099)lp2000,645000.00,531159.50,0.00,645000.00,No,Yes


### CTIPS

In [57]:
def load_ctips(file: str):
  
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )
    
    df_agency = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="agencies")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    )

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    )

    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_house = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="us_house")
    )
    
    df_senate = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="senate")
    )
    
    df_assembly = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="assembly")
    )
    
    df_project['data_source'] = "CTIPS"
    return df_project, df_agency, df_county, df_district, df_phase, df_award, df_house, df_senate, df_assembly

In [58]:
ctips_project, ctips_agencies, ctips_county, ctips_district, ctips_phase, ctips_award, ctips_house, ctips_senate, ctips_assembly = load_ctips('CTIPS_data.xlsx')

In [59]:
ctips_project = database_unique_project_number(ctips_project, 'ctips_id', 'data_source')

In [60]:
ctips_agencies = add_project_number(ctips_project, ctips_agencies, 'ctips_id')

In [61]:
ctips_county = add_project_number(ctips_project, ctips_county, 'ctips_id')

In [62]:
ctips_district = add_project_number(ctips_project, ctips_district, 'ctips_id')

In [63]:
ctips_phase = add_project_number(ctips_project, ctips_phase, 'ctips_id')

In [64]:
ctips_award = add_project_number(ctips_project, ctips_award, 'ctips_id')

In [65]:
ctips_house = add_project_number(ctips_project, ctips_house, 'ctips_id')

In [66]:
ctips_senate = add_project_number(ctips_project, ctips_senate, 'ctips_id')

In [67]:
ctips_assembly = add_project_number(ctips_project, ctips_assembly, 'ctips_id')

In [68]:
ctips_award.sample()

,project_number,line_year,program,progdesc
38150,20700000779CTIPS,2004,Local Funds,None


In [69]:
ctips_assembly.sample()

,project_number,assembly
3585,10900003088CTIPS,56


In [70]:
ctips_agencies.sample()

,project_number,agency_name,implpaed_agency,implpse_agency,implrw_agency,implcon_agency
7776,10900004425CTIPS,Caltrans,NaN,NaN,NaN,NaN


### State Rail Plan

In [71]:
srp_df = har_utils.load_state_rail_plan()

In [75]:
srp_df = non_database_unique_project_number(srp_df, 'data_source')

In [77]:
srp_df.sample()

,project_time_horizon,project_name,project_description,lead_agency,total_project_cost,srp_region,corridor,sub_corridor_node_1,sub_corridor_node_2,itsp_corridor,project_category,data_source,project_number
195,Mid Term,Orange - Olive Junction Improvements and Wye,"The existing wye consists of a single, uncontrolled track and will require modifications to provide PTC-ready track and signal systems. A new crossover will need to be constructed west of the existing wye. Existing ties will be replaced with concrete ties. New control points will be installed. Grade crossings will need to be upgraded to meet quiet zone requirements. A drainage system that includes grading and new catch basins may be necessary pending further preliminary investigation.",Metrolink,32635000,Southern California Megaregion,CA High Speed Rail Phase 1 Corridor,Los Angeles,Anaheim,San Jose/San Francisco Bay Area - Sacramento - Northern Nevada,Capital,State Rail Plan,State_Rail_Plan196


In [78]:
srp_df_agency = separate_out_df(srp_df, ['project_number', 'lead_agency'])

In [79]:
srp_df = srp_df.drop(columns = ['lead_agency'])